In [1]:
import requests as req
import lxml.html
import json
import re, os

In [21]:
def get_song_tags(artist = "", track = ""):
    tmp_artist = artist.lower().replace(' ', '+')
    tmp_track = track.lower().replace(' ', '+')
    url = f'https://www.last.fm/music/{tmp_artist}/_/{tmp_track}'
    html_from_last_fm = req.get(url).text
    html = lxml.html.fromstring(html_from_last_fm)
    track_dict = {'artist': artist, 'track': track, 'tags':[]}
    for li in html.xpath("//ul[@class='tags-list tags-list--global']/li"):
        track_dict['tags'].append(li.text_content())
    return track_dict

In [25]:
def get_track_list(path):
    for file in os.listdir(path):
        track_info = re.split('[-]', file[:-4])
        if track_info[0][-1] == ' ':
            track_info[0] = track_info[0][:-1]
        if track_info[1][0] == ' ':
            track_info[1] = track_info[1][1:]
        yield track_info[0], track_info[1], file

In [28]:
def make_json(path, user_id):
    info = []
    #TODO добавить проверку на наличие тегов. Если нет тегов дальше ничего не делаем
    for artist, track, file_name in get_track_list(path):
        meta = get_song_tags(artist= artist, track= track)
        meta['file_name'] = file_name
        info.append(meta)
    with open(path+f'{user_id}_info.json', 'w') as fp:
        json.dump(info, fp)

In [29]:
make_json('.//tracks//', 1)